In [8]:
!pip install kagglehub pandas scikit-learn fairlearn aif360 --quiet

In [15]:
# Imports
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Load dataset
df = pd.read_csv(r"C:\Users\Arhamsoft\Desktop\Talha Talib Thesis\german_credit_data.csv")

# Drop missing values
df.dropna(inplace=True)

# Show current target values
print("Unique values in 'kredit':", df['kredit'].unique())

# Target and features
y = df['kredit']  # Already binary: 0 = Good, 1 = Bad
X = df.drop(columns=['kredit'])

# Encode categorical features
X_encoded = X.copy()
for col in X_encoded.select_dtypes(include='object').columns:
    X_encoded[col] = LabelEncoder().fit_transform(X_encoded[col])

# Scale numeric features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_encoded)

# Split into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y, test_size=0.3, random_state=42, stratify=y
)

# Train Logistic Regression model
model = LogisticRegression(max_iter=2000, solver='saga')
model.fit(X_train, y_train)

# Make predictions
y_pred = model.predict(X_test)

# Evaluate metrics
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

# Output results
print("\nLogistic Regression Performance Metrics:")
print(f"Accuracy : {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall   : {recall:.4f}")
print(f"F1 Score : {f1:.4f}")


Unique values in 'kredit': [1 0]

Logistic Regression Performance Metrics:
Accuracy : 0.7833
Precision: 0.8033
Recall   : 0.9143
F1 Score : 0.8552


In [17]:
from aif360.datasets import BinaryLabelDataset
from aif360.algorithms.preprocessing import Reweighing
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from aif360.metrics import ClassificationMetric
from sklearn.model_selection import train_test_split
import pandas as pd

# 1. Derive gender from 'famges': 0 = female, 1 = male
df['sex_binary'] = df['famges'].apply(lambda x: 0 if x == 4 else 1)

# 2. Recombine features and target with gender
full_data = pd.concat([X_encoded, y.rename('target'), df['sex_binary']], axis=1)

# 3. Create BinaryLabelDataset for AIF360
bld = BinaryLabelDataset(
    df=full_data,
    label_names=['target'],
    protected_attribute_names=['sex_binary'],
    favorable_label=1,  # 1 = good
    unfavorable_label=0
)

# 4. Split into train/test sets
train_bld, test_bld = bld.split([0.7], shuffle=True)

# 5. Apply reweighing
rw = Reweighing(
    privileged_groups=[{'sex_binary': 1}],
    unprivileged_groups=[{'sex_binary': 0}]
)
train_rw_bld = rw.fit_transform(train_bld)

# 6. Prepare inputs
X_train_rw = train_rw_bld.features
y_train_rw = train_rw_bld.labels.ravel()
w_train_rw = train_rw_bld.instance_weights

X_test_rw = test_bld.features
y_test_rw = test_bld.labels.ravel()

# 7. Train weighted logistic regression
model_rw = LogisticRegression(max_iter=2000, solver='saga')
model_rw.fit(X_train_rw, y_train_rw, sample_weight=w_train_rw)

# 8. Predict
y_pred_rw = model_rw.predict(X_test_rw)

# 9. Performance metrics
acc = accuracy_score(y_test_rw, y_pred_rw)
prec = precision_score(y_test_rw, y_pred_rw)
rec = recall_score(y_test_rw, y_pred_rw)
f1 = f1_score(y_test_rw, y_pred_rw)

print("Reweighed Logistic Regression Performance:")
print(f"Accuracy : {acc:.4f}")
print(f"Precision: {prec:.4f}")
print(f"Recall   : {rec:.4f}")
print(f"F1 Score : {f1:.4f}")

# 10. Fairness metrics
pred_bld_rw = test_bld.copy()
pred_bld_rw.labels = y_pred_rw.reshape(-1, 1)

metric_rw = ClassificationMetric(
    test_bld,
    pred_bld_rw,
    unprivileged_groups=[{'sex_binary': 0}],
    privileged_groups=[{'sex_binary': 1}]
)

spd = metric_rw.statistical_parity_difference()
eod = metric_rw.equal_opportunity_difference()
dir_ = metric_rw.disparate_impact()

print("\nFairness Metrics (Reweighing):")
print(f"Statistical Parity Difference : {spd:.4f}")
print(f"Equal Opportunity Difference : {eod:.4f}")
print(f"Disparate Impact Ratio       : {dir_:.4f}")


Reweighed Logistic Regression Performance:
Accuracy : 0.7233
Precision: 0.7230
Recall   : 0.9953
F1 Score : 0.8376

Fairness Metrics (Reweighing):
Statistical Parity Difference : 0.0148
Equal Opportunity Difference : 0.0051
Disparate Impact Ratio       : 1.0150


C:\Users\Arhamsoft\anaconda3\envs\tf-env\Lib\site-packages\sklearn\linear_model\_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


In [18]:
import tensorflow.compat.v1 as tf
from aif360.algorithms.inprocessing import AdversarialDebiasing
from aif360.datasets import BinaryLabelDataset
from aif360.metrics import ClassificationMetric
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Disable eager execution
tf.disable_eager_execution()
tf.reset_default_graph()

# Session
sess = tf.Session()

# Use 'famges' as gender proxy (already known)
df['sex_binary'] = df['famges'].map({1: 1, 2: 0, 3: 1, 4: 0})  # 1 = male, 0 = female

# Merge features with target and protected attribute
X_encoded['sex_binary'] = df['sex_binary'].values
full_data = pd.concat([X_encoded, y.rename("target")], axis=1)

# Construct BinaryLabelDataset
bld = BinaryLabelDataset(
    favorable_label=1,
    unfavorable_label=0,
    df=full_data,
    label_names=["target"],
    protected_attribute_names=["sex_binary"]
)

# Train/test split using AIF360
train_bld, test_bld = bld.split([0.7], shuffle=True)

# Train model
adv = AdversarialDebiasing(
    privileged_groups=[{'sex_binary': 1}],
    unprivileged_groups=[{'sex_binary': 0}],
    scope_name='adv_debiasing',
    sess=sess,
    num_epochs=50
)
adv.fit(train_bld)

# Predict
pred_bld = adv.predict(test_bld)

# Get labels
y_pred = pred_bld.labels.ravel()
y_true = test_bld.labels.ravel()

# Performance
accuracy = accuracy_score(y_true, y_pred)
precision = precision_score(y_true, y_pred)
recall = recall_score(y_true, y_pred)
f1 = f1_score(y_true, y_pred)

print("Adversarial Debiasing Logistic Regression Performance:")
print(f"Accuracy : {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall   : {recall:.4f}")
print(f"F1 Score : {f1:.4f}")

# Fairness Metrics
metrics = ClassificationMetric(
    test_bld,
    pred_bld,
    privileged_groups=[{'sex_binary': 1}],
    unprivileged_groups=[{'sex_binary': 0}]
)
spd = metrics.statistical_parity_difference()
eod = metrics.equal_opportunity_difference()
dir_ratio = metrics.disparate_impact()

print("\nFairness Metrics (Adversarial Debiasing):")
print(f"Statistical Parity Difference : {spd:.4f}")
print(f"Equal Opportunity Difference : {eod:.4f}")
print(f"Disparate Impact Ratio       : {dir_ratio:.4f}")

sess.close()


Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


epoch 0; iter: 0; batch classifier loss: 117.971664; batch adversarial loss: 0.749353
epoch 1; iter: 0; batch classifier loss: 61.493839; batch adversarial loss: 0.926392
epoch 2; iter: 0; batch classifier loss: 71.515488; batch adversarial loss: 0.910685
epoch 3; iter: 0; batch classifier loss: 86.767815; batch adversarial loss: 0.921624
epoch 4; iter: 0; batch classifier loss: 39.154564; batch adversarial loss: 0.929379
epoch 5; iter: 0; batch classifier loss: 60.244118; batch adversarial loss: 0.881697
epoch 6; iter: 0; batch classifier loss: 64.902222; batch adversarial loss: 0.801165
epoch 7; iter: 0; batch classifier loss: 47.376999; batch adversarial loss: 0.827476
epoch 8; iter: 0; batch classifier loss: 41.744339; batch adversarial loss: 0.761355
epoch 9; iter: 0; batch classifier loss: 60.058620; batch adversarial loss: 0.859334
epoch 10; iter: 0; batch classifier loss: 49.361801; batch adversarial loss: 0.859159
epoch 11; iter: 0; batch classifier loss: 32.999069; batch adve

In [19]:
from aif360.algorithms.postprocessing import EqOddsPostprocessing
from aif360.datasets import BinaryLabelDataset
from aif360.metrics import ClassificationMetric
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Reconstruct dataset
X_encoded['sex_binary'] = df['sex_binary'].values
full_data = pd.concat([X_encoded, y.rename("target")], axis=1)

bld = BinaryLabelDataset(
    favorable_label=1,
    unfavorable_label=0,
    df=full_data,
    label_names=["target"],
    protected_attribute_names=["sex_binary"]
)

train_bld, test_bld = bld.split([0.7], shuffle=True)

# Train baseline model (logistic regression)
model = LogisticRegression(max_iter=1000)
model.fit(train_bld.features, train_bld.labels.ravel())
pred_probs = model.predict_proba(test_bld.features)[:, 1]
pred_labels = (pred_probs >= 0.5).astype(int)

# Wrap prediction into BinaryLabelDataset
test_pred = test_bld.copy()
test_pred.labels = pred_labels.reshape(-1, 1)

# Equalized Odds Postprocessing
eq_odds = EqOddsPostprocessing(
    privileged_groups=[{'sex_binary': 1}],
    unprivileged_groups=[{'sex_binary': 0}]
)
eq_odds = eq_odds.fit(test_bld, test_pred)
eq_pred = eq_odds.predict(test_pred)

# Extract predictions
y_pred_eq = eq_pred.labels.ravel()
y_true_eq = test_bld.labels.ravel()

# Performance metrics
accuracy = accuracy_score(y_true_eq, y_pred_eq)
precision = precision_score(y_true_eq, y_pred_eq)
recall = recall_score(y_true_eq, y_pred_eq)
f1 = f1_score(y_true_eq, y_pred_eq)

print("Equalized Odds Postprocessing Logistic Regression Performance:")
print(f"Accuracy : {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall   : {recall:.4f}")
print(f"F1 Score : {f1:.4f}")

# Fairness metrics
metric = ClassificationMetric(
    test_bld,
    eq_pred,
    privileged_groups=[{'sex_binary': 1}],
    unprivileged_groups=[{'sex_binary': 0}]
)
spd = metric.statistical_parity_difference()
eod = metric.equal_opportunity_difference()
dir_ratio = metric.disparate_impact()

print("\nFairness Metrics (Equalized Odds Postprocessing):")
print(f"Statistical Parity Difference : {spd:.4f}")
print(f"Equal Opportunity Difference : {eod:.4f}")
print(f"Disparate Impact Ratio       : {dir_ratio:.4f}")


Equalized Odds Postprocessing Logistic Regression Performance:
Accuracy : 0.7000
Precision: 0.7339
Recall   : 0.8593
F1 Score : 0.7917

Fairness Metrics (Equalized Odds Postprocessing):
Statistical Parity Difference : 0.0081
Equal Opportunity Difference : 0.0089
Disparate Impact Ratio       : 1.0105


C:\Users\Arhamsoft\anaconda3\envs\tf-env\Lib\site-packages\sklearn\linear_model\_logistic.py:473: ConvergenceWarning: lbfgs failed to converge after 1000 iteration(s) (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT

Increase the number of iterations to improve the convergence (max_iter=1000).
You might also want to scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
